In [220]:
import requests
import pandas as pd

api_key = 'YF0US57AOZXKGIGD'



In [221]:
class script_data:
    def __init__(self, symbol):
        self.symbol = symbol
        # self.interval = interval
        self.url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={api_key}'

    def fetch_intraday_data(self):
        response = requests.get(self.url)
        if response.status_code == 200:
            d = response.json()
            del d['Meta Data']
            return d
        else:
            raise Exception(
                f"Failed to fetch data. Response code: {response.status_code}")
    
    def convert_intraday_data(self, script):
        data = {}
        data["sno"] = []
        data["timestamp"] = []
        data["open"] = []
        data["high"] = []
        data["low"] = []
        data["close"] = []
        data["volume"] = []
        index = 1
        for key, value in script.items():
            if key.startswith('Time Series (5min)'):
                for key, prices in value.items():
                    data["sno"].append(index)
                    index+=1
                    data["timestamp"].append(str(key))
                    data["open"].append(float(prices["1. open"]))
                    data["high"].append(float(prices["2. high"]))
                    data["low"].append(float(prices["3. low"]))
                    data["close"].append(float(prices["4. close"]))
                    data["volume"].append(int(prices["5. volume"]))

        df = pd.DataFrame(data)

        return df
    
    def __getitem__(self, item):
        return self.df[item]
    
    def __setitem__(self, key, value):
        self.df[key] = value
    
    def __contains__(self, item):
        return item in self.df.columns


    def indicator1(df, timeperiod):
        result = df.copy()
        result["indicator"] = result["close"].rolling(window=timeperiod).mean()
        return result[["timestamp", "indicator"]]
    
        


In [222]:
sd = script_data(symbol="GOOGL")
data = sd.fetch_intraday_data()
print(data)


{'Time Series (5min)': {'2023-02-13 20:00:00': {'1. open': '94.3500', '2. high': '94.4000', '3. low': '94.2800', '4. close': '94.4000', '5. volume': '13261'}, '2023-02-13 19:55:00': {'1. open': '94.3500', '2. high': '94.4000', '3. low': '94.3500', '4. close': '94.3500', '5. volume': '6407'}, '2023-02-13 19:50:00': {'1. open': '94.2800', '2. high': '94.3900', '3. low': '94.2800', '4. close': '94.3500', '5. volume': '2081'}, '2023-02-13 19:45:00': {'1. open': '94.4200', '2. high': '94.4400', '3. low': '94.2900', '4. close': '94.2900', '5. volume': '2133'}, '2023-02-13 19:40:00': {'1. open': '94.4700', '2. high': '94.4700', '3. low': '94.3500', '4. close': '94.3500', '5. volume': '917'}, '2023-02-13 19:35:00': {'1. open': '94.3000', '2. high': '94.3400', '3. low': '94.2900', '4. close': '94.3400', '5. volume': '1474'}, '2023-02-13 19:30:00': {'1. open': '94.2800', '2. high': '94.3000', '3. low': '94.2800', '4. close': '94.3000', '5. volume': '609'}, '2023-02-13 19:25:00': {'1. open': '94.

In [223]:
df = sd.convert_intraday_data(data)
print(df)


    sno            timestamp     open   high    low    close  volume
0     1  2023-02-13 20:00:00  94.3500  94.40  94.28  94.4000   13261
1     2  2023-02-13 19:55:00  94.3500  94.40  94.35  94.3500    6407
2     3  2023-02-13 19:50:00  94.2800  94.39  94.28  94.3500    2081
3     4  2023-02-13 19:45:00  94.4200  94.44  94.29  94.2900    2133
4     5  2023-02-13 19:40:00  94.4700  94.47  94.35  94.3500     917
..  ...                  ...      ...    ...    ...      ...     ...
95   96  2023-02-13 12:05:00  94.2900  94.31  94.22  94.2900  398928
96   97  2023-02-13 12:00:00  94.2701  94.31  94.24  94.3000  467125
97   98  2023-02-13 11:55:00  94.2500  94.30  94.12  94.2725  553289
98   99  2023-02-13 11:50:00  94.3064  94.36  94.23  94.2500  503563
99  100  2023-02-13 11:45:00  94.0300  94.41  94.02  94.3021  611545

[100 rows x 7 columns]


In [224]:
sd.df = df
print("open" in sd)  # Output: True


True


In [225]:


result = indicator1(df, timeperiod=5)
print(result)


              timestamp  indicator
0   2023-02-13 20:00:00        NaN
1   2023-02-13 19:55:00        NaN
2   2023-02-13 19:50:00        NaN
3   2023-02-13 19:45:00        NaN
4   2023-02-13 19:40:00   94.34800
..                  ...        ...
95  2023-02-13 12:05:00   94.37956
96  2023-02-13 12:00:00   94.34656
97  2023-02-13 11:55:00   94.31106
98  2023-02-13 11:50:00   94.28206
99  2023-02-13 11:45:00   94.28292

[100 rows x 2 columns]


In [226]:
print(df.columns)


Index(['sno', 'timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
